In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
import random

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

<p style="border:3px #ff0000  solid;">
Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями.

<p style="border:3px #ff0000  solid;">
1. Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:

In [ ]:
movies_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.7/movies.csv')
rating_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.7/ratings_df_sample_2.csv')

In [ ]:
#Для удобства отмасштабируем идентификаторы фильмов таким образом, 
# чтобы они начинались с 0 и заканчивались на n_movies-1
movies_values = rating_df['movieId'].unique()

rating_df['movieId'] = rating_df['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

In [ ]:
#Также отмасштабируем идентификаторы пользователей таким образом, 
# чтобы они начинались с 0 и заканчивались на n_users-1
users_values = rating_df['userId'].unique()

rating_df['userId'] = rating_df['userId'].apply(lambda f: np.where(users_values == f)[0][0])

<p style="border:3px #ff0000  solid;">
2. Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

In [ ]:
rating_df['is_viewed'] = 1

<p style="border:3px #ff0000  solid;">
3. А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать. 

    Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

In [ ]:
users_id_uniq = rating_df['userId'].unique()

In [ ]:
movies_id_uniq = rating_df['movieId'].unique()

    С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм

    Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

In [ ]:
random_pars = []
for i in range(6040099*2):
    para = (random.choice(users_id_uniq), random.choice(movies_id_uniq))
    random_pars.append(para)

In [ ]:
random_pars_df = pd.DataFrame(random_pars)
random_pars_df.columns = ['userId', 'movieId']
random_pars_df.shape


    Среди сгенерированных пар могут быть и дубликаты, удалите их.

In [ ]:
random_pars_df.drop_duplicates(inplace=True)
random_pars_df.shape

    Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

In [ ]:
origin_pars = rating_df[['userId', 'movieId']].copy()

In [ ]:
random_pars_df = set(tuple(x) for x in random_pars_df.values)
origin_pars = set(tuple(x) for x in origin_pars.values)

In [ ]:
random_pars_df_not_viewed = pd.DataFrame(random_pars_df.difference(origin_pars))

In [ ]:
random_pars_df_not_viewed.shape

    Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

In [ ]:
random_pars_df_not_viewed = random_pars_df_not_viewed.sample(6040099)
random_pars_df_not_viewed.columns = ['userId', 'movieId']

    Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

In [ ]:
rating_df = pd.concat([rating_df, random_pars_df_not_viewed], ignore_index=True)
rating_df

In [ ]:
rating_df[['userId', 'movieId']].duplicated().sum()

In [ ]:
rating_df.fillna(0, inplace=True)
rating_df

<p style="border:3px #ff0000  solid;">
4. Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

In [ ]:
train, test = train_test_split(rating_df, test_size=0.1, random_state=123)

In [ ]:
train_target = train['is_viewed']
train_features = train.drop(['is_viewed'], axis=1)
test_target = test['is_viewed']
test_features = test.drop(['is_viewed'], axis=1)

<p style="border:3px #ff0000  solid;">
5. Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

In [ ]:
def dum_model(features):
    return [random.random() for i in range(len(features))]

In [ ]:
dum_predict = dum_model(test_features)
roc_auc_dum = round((roc_auc_score(test_target, dum_predict)), 4)

In [ ]:
rating_df_result = {'ROC-AUC':[roc_auc_dum]}
rating_df_result = pd.DataFrame(rating_df_result)
rating_df_result = rating_df_result.rename(index={0:'dummy_model'})
rating_df_result

<p style="border:3px #ff0000  solid;">
6. Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

In [ ]:
n_users = train['userId'].nunique()
n_users

In [ ]:
n_movies = train['movieId'].nunique()
n_movies

In [ ]:
train_data_matrix = np.array(pd.pivot_table(train, values='is_viewed', index='userId', columns='movieId', fill_value=0))

In [ ]:
train_data_matrix.shape

In [ ]:
train_data_matrix

<p style="border:3px #00B344  solid;">
Рассчитаем попарное косинусное расстояние для пользователей и для фильмов. 
</p>

In [ ]:
user_similarity = cosine_distances(train_data_matrix)

In [ ]:
user_similarity

In [ ]:
movie_similarity = cosine_distances(train_data_matrix.T)

In [ ]:
movie_similarity

In [ ]:
user_similarity.shape

In [ ]:
movie_similarity.shape

<p style="border:3px #00B344  solid;">
Реализуем алгоритм user-based. Для каждого пользователя найдем топ 10 ближайших, исключая самого себя. 
</p>

In [ ]:
top_user_similarity = []
for i in range(n_users):
    neighbors = (user_similarity[i]).argsort()[1:10+1]
    top_user_similarity.append(
        train_data_matrix[neighbors]
    )
top_user_similarity = np.array(top_user_similarity)

In [ ]:
top_user_similarity.shape

Составим матрицу предсказаний, оценив популяронсть фильмов среди пользователей и их соседей относительно других фильмов, посчитав среднее значение просмотров в топ-10.

In [ ]:
predict_movie_popular = top_user_similarity.mean(1)

In [ ]:
predict_movie_popular.shape

<p style="border:3px #00B344  solid;">
Предсказание для тестового датасета 
</p>

In [ ]:
def round_to_near(x):
    return round(x*2)/2

In [ ]:
test['predict_user_based'] = test.apply(
    lambda f: round_to_near(predict_movie_popular[int(f['userId']), int(f['movieId'])]), axis = 1
)

In [ ]:
test.head()

In [ ]:
prediction = np.where(test['predict_user_based'] >= 0.5, 1, 0)
roc_auc_user_based = round(roc_auc_score(prediction, test['is_viewed']), 4)

In [ ]:
rating_df_result.loc[len(rating_df_result.index)] = [roc_auc_user_based]
rating_df_result = rating_df_result.rename(index={1:'user_based'})
rating_df_result

<p style="border:3px #00B344  solid;">
Реализуем алгоритм item-based.
</p>

In [ ]:
top_movie_similarity = []
for i in range(n_movies):
    neighbors_m = (movie_similarity[i]).argsort()[1:10+1]
    top_movie_similarity.append(
        train_data_matrix.T[neighbors_m]
    )
top_movie_similarity = np.array(top_movie_similarity)

In [ ]:
top_movie_similarity.shape

Для каждого пользователя и каждого фильма считаем среднее значение популярности от пользователя для топ-10 соседних фильмов.

In [ ]:
predict_mov_viewed_item_based = top_movie_similarity.mean(1).T

In [ ]:
predict_mov_viewed_item_based.shape

Сделаем предсказание для тестового датасета

In [ ]:
test['predict_movie_based'] = test.apply(
    lambda f: round_to_near(predict_mov_viewed_item_based[int(f['userId']), int(f['movieId'])]), axis = 1
)

In [ ]:
test.head()

In [ ]:
prediction = np.where(test['predict_movie_based'] >= 0.5, 1, 0)
roc_auc_item_based = round(roc_auc_score(prediction, test['is_viewed']), 4)

In [ ]:
rating_df_result.loc[len(rating_df_result.index)] = [roc_auc_item_based]
rating_df_result = rating_df_result.rename(index={2:'item_based'})
rating_df_result

<p style="border:3px #00B344  solid;">
Реализуем алгоритм на основе матричной факторизации. Сделаем SVD.
</p>

In [ ]:
train_data_matrix = train_data_matrix.astype(float)

In [ ]:
u, s, vh = svds(train_data_matrix, k=40)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T

In [ ]:
np.diag(s).shape, users.shape, items.shape

In [ ]:
test['predict_svds'] = test.apply(
    lambda f: round_to_near(np.dot(users[int(f['userId'])], items[int(f['movieId'])])), axis = 1
)

In [ ]:
test.head()

In [ ]:
prediction = np.where(test['predict_svds'] >= 0.5, 1, 0)
roc_auc_svd = round(roc_auc_score(prediction, test['is_viewed']), 4)

In [ ]:
rating_df_result.loc[len(rating_df_result.index)] = [roc_auc_svd]
rating_df_result = rating_df_result.rename(index={3:'svd'})
rating_df_result

<p style="border:3px #ff0000  solid;">
7. Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.

<p style="border:3px #00B344  solid;">
Вывод: Исходя из полученных значений метрики ROC-AUC для каждой модели, можно сделать вывод, что наилучшим качеством обладает модель на основе матричной факторизации.
</p>